imports
-

In [1]:
import sys, os
sys.version

'3.9.13 (main, Aug 25 2022, 23:26:10) \n[GCC 11.2.0]'

In [2]:
#math
import numpy as np
#capture
import cv2
import wave
#utils
from tqdm import tqdm
from IPython.display import Image
import datetime
from collections import deque
import matplotlib.pyplot as plt

In [3]:
%load_ext tensorboard

In [4]:
#neural
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
print(tf.__version__)

2.9.2


In [5]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.compat.v1.Session(config=config)

2022-10-23 15:36:43.742994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 15:36:44.445380: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 599 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


system info
-

In [6]:
for i in device_lib.list_local_devices():
    print(f'{i.device_type}|{i.name} memory limit: {i.memory_limit}')
    print(i.physical_device_desc)

CPU|/device:CPU:0 memory limit: 268435456

GPU|/device:GPU:0 memory limit: 628693401
device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


2022-10-23 15:36:44.451357: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:0 with 599 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


import data
-

In [7]:
def reshape(frame):
    global WIDTH, HEIGHT
    newframe = np.zeros(WIDTH*HEIGHT).reshape(HEIGHT, WIDTH)
    if frame.shape[0] == HEIGHT and frame.shape[1] == WIDTH:
        return frame
    elif frame.shape[0] <= HEIGHT and frame.shape[1] <= WIDTH:
        newframe[:frame.shape[0],:frame.shape[1]] = frame
    elif frame.shape[0] <= HEIGHT:
        newframe[:frame.shape[0],:] = frame[:, :WIDTH]
    elif frame.shape[1] <= WIDTH:
        newframe[:,:frame.shape[1]] = frame[:HEIGHT, :]
    else:
        newframe = frame[:HEIGHT, :WIDTH]
    return newframe

def deinterlase(frame):
    even_fields=frame[::2,:];
    odd_fields=frame[1::2,:];
    return even_fields, odd_fields

In [8]:
WIDTH = int(720)
HEIGHT = int(625-1)

def normilizeFrame(frame):
    global WIDTH, HEIGHT
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #change dimentions(choose 1 of 2 options)
    #frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    frame = reshape(frame)
    
    return deinterlase(frame)
    #return frame / 255.0

def retrieve():
    global WIDTH, HEIGHT
    if cap.grab():
        ret, frame = cap.retrieve()
        if not ret:
            print('error reading frame')
            return False, normilizeFrame(np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT))
        return True, normilizeFrame(frame)
    else:
        return False, normilizeFrame(np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT))

def getNext():
    global first, bottomFrame, half
    if first == True:
        first = False
        half = False
        ret, (topFrame, bottomFrame) = retrieve()
        return ret, topFrame[np.newaxis is None,:,:]
    
    if half:
        ret, (topFrame, bottomFrame) = retrieve()
        half = False
        return ret, topFrame[np.newaxis is None,:,:]
    else:
        half = True
        return True, bottomFrame[np.newaxis is None,:,:]

def openCap(file):
    global cap, gray, half, first
    first = True
    cap = cv2.VideoCapture(file + '.avi')
    if (cap.isOpened()== False): 
      print("Error opening video stream or file")
openCap('./dataset/video')

In [9]:
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(frameCount)

#tech calc

#audio in pcm have 44100 samples in second per channel, respectively we have 88200 samples
#per frame we have 44100/25(pal) = 1764 samples(for single channel)(and we take half of frame for learning /2)

samplesPerFrame = int(44100/cap.get(cv2.CAP_PROP_FPS)/2)
print(samplesPerFrame)

1501
882


create dataset

In [10]:
dir = "./dataset/"

In [11]:
def initQueues():
    global bufR0, bufL1, bufR1, bufR2, bufL2
    bufR0 = deque()
    bufL1 = deque()
    bufR1 = deque()
    bufL2 = deque()
    bufR2 = deque()
    
    for i in range(1*16):
        bufR0.append(0)
    
    for i in range(2*16):
        bufL1.append(0)
    
    for i in range(3*16):
        bufR1.append(0)
    
    for i in range(4*16):
        bufL2.append(0)
    
    for i in range(5*16):
        bufR2.append(0)    
        
def pairwise(iterable):
    a = iter(iterable)
    if len(iterable) % 6 != 0:
        print(f'input length detected that is not divisible by 6! all data after the {len(iterable)-(len(iterable) % 6)}({len(iterable) % 6} cells) position will be ignored.')
    return zip(a, a, a, a, a, a)
    
def Abreacker(sample):
    global bufR0, bufL1, bufR1, bufR2, bufL2
    filtered = np.array([], dtype=np.uint16)
    line = np.zeros(6, dtype=np.uint16)

    for elem in pairwise(sample):
        bufR0.append(elem[1])
        bufL1.append(elem[2])
        bufR1.append(elem[3])
        bufL2.append(elem[4])
        bufR2.append(elem[5])
        
        line[0] = elem[0]
        line[1] = bufR0.popleft()
        line[2] = bufL1.popleft()
        line[3] = bufR1.popleft()
        line[4] = bufL2.popleft()
        line[5] = bufR2.popleft()
                
        filtered = np.append(filtered, line)
    
    return filtered

In [12]:
def initQueuesForDebreacker():
    global bufL0D, bufR0D, bufL1D, bufR1D, bufL2D
    bufL0D = deque()
    bufR0D = deque()
    bufL1D = deque()
    bufR1D = deque()
    bufL2D = deque()
    
    for i in range(5*16):
        bufL0D.append(0)
        
    for i in range(4*16):
        bufR0D.append(0)
    
    for i in range(3*16):
        bufL1D.append(0)
    
    for i in range(2*16):
        bufR1D.append(0)
    
    for i in range(1*16):
        bufL2D.append(0)
        
def Adebreacker(sample):
    global bufL0D, bufR0D, bufL1D, bufR1D, bufL2D
    filtered = np.array([], dtype=np.uint16)
    line = np.zeros(6, dtype=np.uint16)
    
    for elem in pairwise(sample):
        bufL0D.append(elem[0])
        bufR0D.append(elem[1])
        bufL1D.append(elem[2])
        bufR1D.append(elem[3])
        bufL2D.append(elem[4])
        
        line[0] = bufL0D.popleft()
        line[1] = bufR0D.popleft()
        line[2] = bufL1D.popleft()
        line[3] = bufR1D.popleft()
        line[4] = bufL2D.popleft()
        line[5] = elem[5]
                
        filtered = np.append(filtered, line)
    
    return filtered


In [13]:
mainAudioFile = wave.open(f'{dir}audio.wav', mode='r')

def gen():
    global mainAudioFile, samplesPerFrame
    
    #audio
    initQueues()
    mainAudioFile.rewind()
    
    if not (mainAudioFile.getparams()[0] == 2 and mainAudioFile.getparams()[2] == 44100):
        print('wrong audiofile params')
        sys.exit(1)
    
    
    #video
    openCap(f'{dir}video')
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT) -1
    
    
    while int(cap.get(cv2.CAP_PROP_POS_FRAMES)) <= frames:
        #audio
        frame = mainAudioFile.readframes(samplesPerFrame)
        
        if len(frame) < samplesPerFrame*2:
            frame = frame + b'\x00' * ((samplesPerFrame*4)-len(frame))
        
        #video
        vframe = getNext()[1]
        
        yield vframe, (Abreacker(tf.io.decode_raw(frame, tf.uint16, little_endian=True).numpy())/65535)

        
dataset=tf.data.Dataset.from_generator(
    gen, 
    output_signature=(tf.TensorSpec(shape=(1, int(HEIGHT/2), WIDTH), dtype=tf.uint8, name='image'),
                      tf.TensorSpec(shape=(samplesPerFrame*2,), dtype=tf.float32, name='audio'),
                      #tf.TensorSpec(shape=(1), dtype=tf.int32, name='frameNum')
                     )
)#.cache('./dataset/')

2022-10-23 15:36:44.514688: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 599 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


In [14]:
list(dataset.take(1))[0][1].numpy().max()

0.9987335

#create model
-

In [15]:
input = keras.Input(shape=(int(HEIGHT/2), WIDTH), name = 'image')

In [16]:
x = tf.keras.layers.Rescaling(1./255, name='normalize')(input)
i = 1
x = tf.keras.layers.Conv1D(360, 1, activation='sigmoid', name=f'conv1d_{i}')(x)
#i +=1
#x = tf.keras.layers.Conv1D(360, 1, activation='sigmoid', name=f'conv1d_{i}')(x)

print(x.shape)

x = tf.keras.layers.Reshape((x.shape[1], x.shape[2], 1))(x)
#
i = 1
x = tf.keras.layers.Conv2D(8, (1, 4), strides=(1,2), activation='relu', name=f'conv2d_{i}')(x)
i +=1
x = tf.keras.layers.Conv2D(8, (1, 2), strides=(1,2), activation='relu', name=f'conv2d_{i}')(x)
i +=1
x = tf.keras.layers.Conv2D(8, (1, 14), strides=(1,14), activation='relu', name=f'conv2d_{i}')(x)
#i +=1
#x = tf.keras.layers.Conv2D(3, 32, activation='sigmoid', name=f'conv2d_{i}')(x)
#
print(x.shape)
print(f'diff total:{x.shape[1]*x.shape[2]*x.shape[3]-(samplesPerFrame*4)}') 

(None, 312, 360)
(None, 312, 6, 8)
diff total:11448


In [17]:
x = tf.keras.layers.Flatten(name='Flatten')(x)

In [18]:
x = tf.keras.layers.Dense((samplesPerFrame*2), activation='relu', use_bias=True, name='audio')(x)

In [19]:
model = keras.Model(input, x)
model.summary()

#tf.keras.utils.plot_model(model, to_file="model.png", show_shapes=True, show_dtype=False, show_layer_names=True, rankdir="TB", expand_nested=True, dpi=96, layer_range=None, show_layer_activations=True)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 312, 720)]        0         
                                                                 
 normalize (Rescaling)       (None, 312, 720)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 312, 360)          259560    
                                                                 
 reshape (Reshape)           (None, 312, 360, 1)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 312, 179, 8)       40        
                                                                 
 conv2d_2 (Conv2D)           (None, 312, 89, 8)        136       
                                                                 
 conv2d_3 (Conv2D)           (None, 312, 6, 8)         904   

compile and learn
-

In [20]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy', 'mean_squared_error', 'MeanAbsolutePercentageError'])
loss = 1
step = 0

In [21]:
!rm -rf ./logs/ ./model/ ./output/
!mkdir logs
!mkdir model
!mkdir output

In [22]:
testfile = 'testfiles/pal/FROZTEN_-_Domestic_Technology'

In [23]:
def predict():
    global step, testfile, vframe, model, cap
    initQueuesForDebreacker()
    
    openCap(testfile)
    bar = tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)*2), ncols=100)
    file = wave.open(f'output/{step}.wav', mode='w')
    file.setnchannels(2) 
    file.setsampwidth(2)
    file.setframerate(44100)
    ret, vframe = getNext()
    frst = True
    
    while ret:
        aframe = Adebreacker((model.predict(vframe, verbose=0, batch_size=64)[0]*65535).astype(np.uint16))
        if frst:
            frst = False
            file.writeframes(aframe[480:].tobytes())
        else:
            file.writeframes(aframe.tobytes())
        bar.update(1)
        ret, vframe = getNext()
        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) > 25*2*20:    #limiting on 20 seconds
            ret = False
    bar.close()
    file.close()

In [24]:
predict()

 20%|███████████▉                                               | 2000/9874 [01:47<07:04, 18.56it/s]


try to generate pcm frame(just for understanding)

In [25]:
while loss >= 0.001:
    step = step +1
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    fitHistory = model.fit(dataset, epochs=10, batch_size=32, callbacks=[tensorboard_callback])
    model.save(f'model/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}', save_format="h5")
    
    predict()
    
    loss = fitHistory.history['loss'][0]

Epoch 1/10
   3000/Unknown - 78s 26ms/step - loss: 0.1439 - binary_accuracy: 6.1625e-05 - mean_squared_error: 0.1439 - mean_absolute_percentage_error: 8596.3789

2022-10-23 15:39:52.038299: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6340239360 exceeds 10% of free system memory.


3001/3001 [==============================] - 83s 27ms/step - loss: 0.1440 - binary_accuracy: 1.1481e-04 - mean_squared_error: 0.1440 - mean_absolute_percentage_error: 168794.7188
Epoch 2/10
3001/3001 [==============================] - ETA: 0s - loss: 0.1028 - binary_accuracy: 3.4215e-04 - mean_squared_error: 0.1028 - mean_absolute_percentage_error: 90546.8828

2022-10-23 15:41:12.386301: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6340239360 exceeds 10% of free system memory.


3001/3001 [==============================] - 79s 26ms/step - loss: 0.1028 - binary_accuracy: 3.4215e-04 - mean_squared_error: 0.1028 - mean_absolute_percentage_error: 90546.8828
Epoch 3/10
3000/3001 [============================>.] - ETA: 0s - loss: 0.0998 - binary_accuracy: 6.0878e-05 - mean_squared_error: 0.0998 - mean_absolute_percentage_error: 27039.0762

...

3001/3001 [==============================] - 79s 26ms/step - loss: 0.0927 - binary_accuracy: 3.4735e-04 - mean_squared_error: 0.0927 - mean_absolute_percentage_error: 166216.9062
Epoch 6/10
3001/3001 [==============================] - 78s 26ms/step - loss: 0.0924 - binary_accuracy: 3.4823e-04 - mean_squared_error: 0.0924 - mean_absolute_percentage_error: 165787.8906
Epoch 7/10
3001/3001 [==============================] - 79s 26ms/step - loss: 0.0923 - binary_accuracy: 3.4842e-04 - mean_squared_error: 0.0923 - mean_absolute_percentage_error: 74530.7422
Epoch 8/10
3001/3001 [==============================] - 80s 27ms/step - loss: 0.0906 - binary_accuracy: 3.4505e-04 - mean_squared_error: 0.0906 - mean_absolute_percentage_error: 26300.9121
Epoch 9/10
3001/3001 [==============================] - 80s 27ms/step - loss: 0.0895 - binary_accuracy: 3.4607e-04 - mean_squared_error: 0.0895 - mean_absolute_percentage_error: 30653.4785
Epoch 10/10
3001/3001 [==============================] - 80s 27m

 20%|███████████▉                                               | 2000/9874 [01:49<07:09, 18.31it/s]

Epoch 1/10


   1214/Unknown - 32s 26ms/step - loss: 0.0887 - binary_accuracy: 1.2722e-04 - mean_squared_error: 0.0887 - mean_absolute_percentage_error: 60178.0352


KeyboardInterrupt



In [ ]:
%tensorboard --logdir logs/fit